In [294]:
# fetching web pages
import requests
# getting/processing html from web pages
from bs4 import BeautifulSoup
# processing text with regular expressions
import re
# dataframes and writing to csv
import pandas as pd

In [286]:
# setup variables
# base url
root='https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/'
# how many stocks are there
numstocks=30
# target tags
targettags=['h3', 'p']
# success code
success=200

In [3]:
# check
for i in range(1, numstocks+1):
    url=root+str(i)
    print (url)

https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/1
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/2
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/3
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/4
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/5
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/6
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/7
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/8
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/9
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q

In [173]:
print (h3,'\n')
# extract text from h3 tag
print (h3.text)

<h3 style="text-align: justify;">9.  PayPal Holdings, Inc. (NASDAQ: <a href="https://www.insidermonkey.com/insider-trading/company/paypal%20holdings%20inc/1633917/">PYPL</a>): $290.81</h3> 

9.  PayPal Holdings, Inc. (NASDAQ: PYPL): $290.81


In [164]:
def parseheader(h3):
    # print (h3)
    # convert to string
    h3text=h3.text
    # print (h3text)
    # grab digits at the front of the string (rank)
    rank=re.search('^\d*', h3text).group()
    # print ("rank:", rank )
    # grab the $ price - dollar, zero or more digits, decimal, zero or more digits
    price=re.search('\$\d*.*\d*', h3text).group().strip('\$')
    # print ("price:", price)
    # grab colon separated exchange followed by ticker
    exchangeticker=re.sub(' ','', re.search('\([A-Z]*: [A-Z]*\)', h3text).group())
    # print ("exchange and ticker:", exchangeticker)
    exchange, ticker=re.sub('\(|\)','', exchangeticker).split(':')
    # print ("ticker:", ticker)
    # print ("exchange:", exchange)
    # google finance compatible ticker
    gfinanceticker=ticker+":"+exchange
    # print ("Google Finance compatible ticker:", gfinanceticker)
    # grab the stock shortname (this is ugly, but regex will probably not work here)
    shortname=h3text.split('(')[0].split('.',1)[1].strip()
    # print (shortname)
    # print (rank, longticker, shortname, price)
    return (rank, gfinanceticker, shortname, price)

In [174]:
rank, gfinanceticker, shortname, price = parseheader(h3)
print (rank, gfinanceticker, shortname, price)

9 PYPL:NASDAQ PayPal Holdings, Inc. 290.81


In [267]:
print (p.contents)

['Number of Hedge Funds: 147 (2020Q4)', '\nNumber of Hedge Funds: 150 (2020Q3)', '\nTotal Dollar Amount of Long Hedge Fund Positions: $15.96 billion', '\nPercent of Hedge Funds with Long Positions: 16.6%', '\nFirst Quarter Return (through February 18): 24.2%', '\nPopularity Ranking (2020Q3): 8', '\nPopularity Ranking (2020Q2): 9', '\nNoteworthy Hedge Fund Shareholders: ', <a href="https://www.insidermonkey.com/hedge-fund/coatue+management/79/">Philippe Laffont</a>, ',\xa0', <a href="https://www.insidermonkey.com/hedge-fund/scge+management/555/">Christopher Lyle</a>, ', ', <a href="https://www.insidermonkey.com/hedge-fund/fisher+asset+management/11/">Fisher Asset Management</a>]


In [281]:
print (p.text)

Number of Hedge Funds: 147 (2020Q4)
Number of Hedge Funds: 150 (2020Q3)
Total Dollar Amount of Long Hedge Fund Positions: $15.96 billion
Percent of Hedge Funds with Long Positions: 16.6%
First Quarter Return (through February 18): 24.2%
Popularity Ranking (2020Q3): 8
Popularity Ranking (2020Q2): 9
Noteworthy Hedge Fund Shareholders: Philippe Laffont, Christopher Lyle, Fisher Asset Management


In [282]:
def parseparagraph(p):
    # print (p)
    # remove <br> tags
    for empty in p.findAll('br'):
      empty.extract()
    # convert to string
    pcontents=p.contents
    # print (pcontents)
    numhedgefunds2020q4 = re.search ('\s\d*\s', pcontents[0]).group()
    numhedgefunds2020q3 = re.search ('\s\d*\s', pcontents[1]).group()
    totaldollarlongpositions = re.search ('\$\d*.\d*', pcontents[2]).group().strip('$')
    percentfundslongpositions = re.search ('\d*.\d*%$', pcontents[3]).group().strip('%')
    latestquarterreturn = re.search ('\d*.\d*%$', pcontents[4]).group().strip('%')
    popranking2020q3 = re.search (' \d*$', pcontents[5]).group().strip()
    popranking2020q2 = re.search (' \d*$', pcontents[6]).group().strip()
    # mumble grumble....
    # noteworthyhedgies = pcontents[7]
    # print (numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2)
    return (numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2)

In [ ]:
def parsepositions(input):
    # tokens=re.split(':|\(|\)', input.strip('\n'))
    tokens=re.split(':|\$|\%', input.replace(u'\xa0', '').strip('\n'))
    # print (tokens)
    positionsname=(tokens[0].replace(" ","-")+'-billions').lower()
    positionsvalue=tokens[2].split()[0]
    return(positionsname, positionsvalue)

In [283]:
numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2=parseparagraph(p)
print (numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2)

 147   150  15.96 16.6 24.2 8 9


In [293]:
for i in range(25, numstocks+1):
    url=root+str(i)
    print (url)
    page=requests.get(url)
    if (page.status_code == success):
        print('Visited URL: {}'.format(page.url))
        html = BeautifulSoup(page.content, 'html.parser')
        # extract the <h3> tag
        h3=html.body.h3
        # extract the <p> tag
        if i==1: # hack for the first page
            p=html.select('p')[18]
        else: p=html.body.p
        rank, gfinanceticker, shortname, price = parseheader(h3)
        numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2=parseparagraph(p)

        print (rank, gfinanceticker, shortname, price)
        print (numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2)
    else:
        print ('Oops, got error: {}', format(page.status_code))
print ("Done!")

https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/25
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/25/
6 GOOG:NASDAQ Alphabet Inc. Class C 2,117.20
 157   150  20.6 17.7 20.9 7 8
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/26
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/26/
5 V:NYSE Visa Inc. 209.35
 166   160  23.6 18.7 4.1 6 6
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/27
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/27/
4 GOOGL:NASDAQ Alphabet Inc. Class A 2,105.81
 179   162  21.9 20.2 20.2 5 4
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/28
Visited URL: https://www.ins